### This notebook clean up beauty product data and prepare some basic statistic features.

In [1]:
from difflib import SequenceMatcher
from ast import literal_eval
from collections import Counter
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import operator
import itertools
import gc
import pickle
import os.path

### Prepare ingredient information

Read ingredient dataframe:

In [2]:
cols = ['name','category','rating']
ingredient_df = pd.read_csv('../web_scraper/ingredients.csv', usecols=cols, converters={"category": literal_eval})
ingredient_df['name'] = ingredient_df['name'].str.strip()
ingredient_df['rating_num'] = ingredient_df['rating'].map({'Poor':0, 'Average':1, 'Good':2, 'GOOD':2, 'Best':3})
print("number of ingredient:",ingredient_df.shape[0])
ingredient_df.head()

number of ingredient: 1750


,name,category,rating,rating_num
0,"1, 2-Hexanediol",[Preservatives],Good,2
1,10-Hydroxydecanoic Acid,[Emollients],Good,2
2,4-T-butylcyclohexanol,"[Emollients, Skin-Soothing]",Good,2
3,Acacia farnesiana extract,"[Plant Extracts, Fragrance: Synthetic and Frag...",Poor,0
4,acacia senegal gum,"[Texture Enhancer, Plant Extracts, Skin-Soothing]",Good,2


Create a class where we can check if an ingredient matches our existing ingredient dictionary. if there is a match,
find the ingredient's rating and category.
* Initialize the class with ingredient rating dictionary and category dictionary.
* Given a list of ingredient, find best matching ingredient that rating and category are avaliable. This is done by evaluating the similarity metric between the name of all existing ingredient and the name of given ingredient (use python function SequenceMatcher). If the similarity is below a thresh, then the given ingredient is labeled as 'unknown'.
* After building up the matching dictionary, we can find an ingredient's matching, rating and category by calling the lookup function.

In [3]:
class look_up_ingredient():
    
    def __init__(self, rating_dict, category_dict):
        self.rating_dict = rating_dict
        self.rating_dict['unknown'] = np.nan
        
        self.category_dict = category_dict
        self.category_dict['unknown'] = []
        
        self.rating = set([value for value in self.rating_dict.values()])
        self.category = set([value for values in self.category_dict.values() for value in values])
        
        self.match_dict = {}
    
    def find_matching_ingredient(self, my_ingredients, thresh=0.25):

        for ingredient in tqdm(my_ingredients):
            if ingredient in self.match_dict.keys():
                continue
            match_matric = {key : SequenceMatcher(None, key, ingredient).ratio() for key in self.rating_dict.keys()}
            best_match, best_metric = max(match_matric.items(), key=operator.itemgetter(1))
            if best_metric > thresh:
                self.match_dict[ingredient] = best_match
            else:
                self.match_dict[ingredient] = 'unknown'
                
    def lookup(self, ingredient, option=''):
        
        key = self.match_dict.get(ingredient, 'unknown')
        rating = self.rating_dict.get(key, -1)
        category = self.category_dict.get(key, [])
        
        if option == 'ingredient':
            return key
        elif option == 'rating':
            return rating
        elif option == 'category':
            return category
        else:
            return key, rating, category
    
    def save_match_dict(self, dictfile="ingredient_match_dict.pickle"):
        pickle_out = open(dictfile,"wb")
        pickle.dump(match, pickle_out)
        pickle_out.close()
        
    def load_match_dict(self, dictfile="ingredient_match_dict.pickle"):
        if os.path.isfile(dictfile):
            pickle_in = open(dictfile,"rb")
            self.match_dict = pickle.load(pickle_in)

Create ingredient class, note for ingredient with alias we will duplicate the record. 

For example, for "PEG/PPG-18/18 dimethicone" we will create three dict items, with different keys but same value.

In [4]:
ingredient_rating_dict = {name: row['rating_num'] for (idx, row) in ingredient_df.iterrows() for name in row['name'].split('/')}
ingredient_category_dict = {name: row['category'] for (idx, row) in ingredient_df.iterrows() for name in row['name'].split('/')}
lookup = look_up_ingredient(ingredient_rating_dict, ingredient_category_dict)

In [5]:
#lookup.save_match_dict()

### Clean product data 

* Drop products that are not "chemical" products, like makeup brushes, cleaning devices.
* Merge some categories.
* Split 'size' column to a number and unit, to unit conversion as necessary
* Compute 'price/size'
* Basic cleaning on ingredients:
    * split inactive and active ingredient
    * convert ingredients to a list
    * find number of inactive and active ingredient
    * check if the ingredients are in alphabatical order -- most companies like to list ingredient in a descending order of their quantity in the product, some companies just list ingredients alphabatically.
* Look up ingredients in our ingredient dictionary.
    * get a set of all unique ingredients in the products dataframe
    * find the match of all these ingredients
    * for all product, we loop over its ingredient list and look up the matching ingredient, rating and ingredient category
    * count how many ingredients in a product have a certain rating (how many ingredient rated as Good/Average etc.)
    * count how many ingredients in a product belongs to a certain category (how many antioxidants/sunscreen etc.)
    * compute average ingredient rating. For inactive ingredient, we also consider two kinds of weighted average.

In [6]:
class product_df_cleaning:
    
    def __init__(self, df):
        self.df = df.copy(deep=True)
    
    def clean_price(self):
        # convert all price to float
        # compute price/size
        if self.df['price'].dtype != 'float':
            self.df['price'] = self.df['price'].apply(lambda x: x.replace(',','')).astype('float')
        self.df['avg_price'] = self.df['price']/self.df['size_num']
        
    def clean_size(self):
        # split size number and unit
        # convert fl.oz. to ml
        self.df['size_num'] = self.df['size'].apply(lambda x: float(str(x).split()[0]))
        self.df['size_unit'] = self.df['size'].apply(lambda x: str(x)[len(str(x).split()[0])+1:])
        self.df['size_num'].loc[self.df['size_unit']=='fl. oz.'] *= 29.5735
        self.df['size_num'] = self.df['size_num'].round()
        self.df['size_unit'].loc[self.df['size_unit']=='fl. oz.'] = 'ml'
        self.df['size_unit'].loc[self.df['size_unit']=='grams'] = 'gram'
        self.df['size_unit'].loc[~self.df['size_unit'].isin(['ml','gram',''])] = 'piece/other'
        
    def clean_ingredient(self):
        
        def split_active_inactive(sr_ingredient):
            inactive_start = pd.concat([sr_ingredient.str.find('Other'),
                                       sr_ingredient.str.find('Inactive'),
                                       sr_ingredient.str.find('Cosmetic Ingredients')],
                                       axis=1).max(axis=1)
            
            inactive_start = inactive_start.replace(-1, 0)
            inactive = [ingredient[start:] for (ingredient, start) in zip(sr_ingredient, inactive_start)]
            inactive = [ingredient[ingredient.find(':')+1:] for ingredient in inactive]                           
            active = [ingredient[:start] for (ingredient, start) in zip(sr_ingredient, inactive_start)]
            active = [ingredient[ingredient.rfind(':')+1:] for ingredient in active]
            return active, inactive
             
        def check_alphabetical(word_list, tol=1):
            if(len(word_list)) <= tol:
                return True
            count = 0
            for i in range(len(word_list) - 1):
                if word_list[i] > word_list[i + 1]:
                    count += 1
                if count > tol:
                    return False
            return True
        
        #split active and inactive ingredient
        self.df['ingredient'].fillna('', inplace=True)
        self.df['active_ingredient'], self.df['inactive_ingredient'] = split_active_inactive(self.df['ingredient'])
        #convert to list
        self.df['active_ingredient_list'] = self.df['active_ingredient'].apply(lambda x: [l.strip() for l in str(x).split(',') if l.lower().islower()])
        self.df['inactive_ingredient_list'] = self.df['inactive_ingredient'].apply(lambda x: [l.strip() for l in str(x).split(',') if l.lower().islower()])      
        #find number of ingredient
        self.df['n_inactive_ingredient'] = self.df['inactive_ingredient_list'].apply(lambda x: len(x))
        self.df['n_active_ingredient'] = self.df['active_ingredient_list'].apply(lambda x: len(x))
        #check if ingredients are listed alphabatically or perhaps by there quantity
        self.df['is_alphabatical'] = self.df['inactive_ingredient_list'].apply(check_alphabetical)

        
    def lookup_ingredients(self, lookup):
        
        print("processing all ingredients...")
        merged_ingredients = set(list(itertools.chain(*self.df['inactive_ingredient_list'].values)))
        merged_ingredients = merged_ingredients.union(
                             set(list(itertools.chain(*self.df['active_ingredient_list'].values))))
        lookup.find_matching_ingredient(merged_ingredients)
        ingredient_property = pd.DataFrame(index=merged_ingredients)
        
        print("find all ingredients information...")
        ingredient_property['matching'] = [lookup.lookup(ingredient, option='ingredient') 
                                           for ingredient in merged_ingredients]
        ingredient_property['rating'] = [lookup.lookup(ingredient, option='rating') 
                                           for ingredient in merged_ingredients]
        ingredient_property['category'] = [lookup.lookup(ingredient, option='category') 
                                           for ingredient in merged_ingredients]
        
        # map origianal ingredient list to matched ingredient
        self.df['inactive_ingredient_matched_list'] = [[ingredient_property.loc[ingredient, 'matching'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['inactive_ingredient_list'].values]
        self.df['active_ingredient_matched_list'] = [[ingredient_property.loc[ingredient, 'matching'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['active_ingredient_list'].values]
        
        # map original ingredient list to ingredient rating
        self.df['inactive_ingredient_rating_list'] = [[ingredient_property.loc[ingredient, 'rating'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['inactive_ingredient_list'].values]
        self.df['active_ingredient_rating_list'] = [[ingredient_property.loc[ingredient, 'rating'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['active_ingredient_list'].values]
        
        # map original ingredient list to ingredient category
        self.df['inactive_ingredient_category_list'] = [[ingredient_property.loc[ingredient, 'category'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['inactive_ingredient_list'].values]
        self.df['active_ingredient_category_list'] = [[ingredient_property.loc[ingredient, 'category'] 
                                                        for ingredient in ingredients]
                                                        for ingredients in self.df['active_ingredient_list'].values]
        
        def count_ingredient(col, prefix='count', mode='rating'):
            count_df = pd.DataFrame()
            if mode=='rating':
                count_df = pd.DataFrame.from_dict([dict(Counter(row))
                                                   for row in self.df[col].values])
            elif mode=='cat':
                # if an ingredient belongs to multiple category, we will increment all categories
                count_df = pd.DataFrame.from_dict([dict(Counter([cat for catlist in row for cat in catlist]))
                                                   for row in self.df[col].values])
            else:
                print('unknown mode in count_ingredient')
                return count_df
            
            count_df.set_index(self.df.index, inplace=True)
            count_df.fillna(0, inplace=True)
            count_df = count_df.add_prefix(prefix)
            return count_df
        
        # count inactive rating
        inactive_rating_count = count_ingredient('inactive_ingredient_rating_list', 
                                                 prefix='inactive_rating_count_')
        if 'inactive_rating_count_nan' in inactive_rating_count:
            inactive_rating_count.drop(['inactive_rating_count_nan'],axis=1,inplace=True) #hack...
        
        # count active rating
        active_rating_count = count_ingredient('active_ingredient_rating_list', 
                                               prefix='active_rating_count_')
        if 'active_rating_count_nan' in active_rating_count:
            active_rating_count.drop(['active_rating_count_nan'],axis=1,inplace=True) #hack... 

        # count inactive category
        inactive_category_count = count_ingredient('inactive_ingredient_category_list', 
                                                   prefix='inactive_cat_count_', mode='cat')
        
        # count active category
        active_category_count = count_ingredient('active_ingredient_category_list', 
                                                 prefix='active_cat_count_', mode='cat')
        
        # merge to main dateframe
        self.df = pd.concat([self.df, 
                             inactive_rating_count,
                               active_rating_count,
                             inactive_category_count,
                               active_category_count,], axis=1)
        
        # compute average/weighted ingredient rating
        def get_mean_rating(x):
            return np.nanmean(x)
        
        def get_weighted_rating(x):
            weight = np.arange(len(x)) + 1
            weight = 1.0 / weight
            weight[np.isnan(x)] = 0
            weight = weight/np.sum(weight)
            return np.nansum(x*weight)
            
        def get_weighted_rating2(x):
            weight = np.arange(len(x))
            weight = np.exp(-weight/10.0)
            weight[np.isnan(x)] = 0
            weight = weight / np.sum(weight)
            return np.nansum(x*weight)
        
        # mean rating
        self.df['active_mean_rating'] = self.df['active_ingredient_rating_list'].apply(get_mean_rating)
        self.df['inactive_mean_rating'] = self.df['inactive_ingredient_rating_list'].apply(get_mean_rating)
        # weighted rating 1: only apply to non-alphabatical ordered
        self.df['inactive_mean_rating_w1'] = self.df['inactive_ingredient_rating_list'].apply(get_weighted_rating)
        self.df['inactive_mean_rating_w1'].loc[self.df['is_alphabatical']] = self.df['inactive_mean_rating']
        # weighted rating 2: only apply to non-alphabatical ordered
        self.df['inactive_mean_rating_w2'] = self.df['inactive_ingredient_rating_list'].apply(get_weighted_rating2)
        self.df['inactive_mean_rating_w2'].loc[self.df['is_alphabatical']] = self.df['inactive_mean_rating']
        
        del ingredient_property
        del inactive_rating_count, active_rating_count
        del inactive_category_count, active_category_count
        gc.collect()
        
    def basic_clean(self):        
        self.clean_size()
        self.clean_price()
        self.clean_ingredient()
        
    def get_df(self):
        return self.df

Read product data from disk

In [7]:
cols = ['product_names','product_category','brand','ingredient','size','price']
skin_care_df = pd.read_csv('../web_scraper/skin_care_products.csv', usecols = cols)
body_care_df = pd.read_csv('../web_scraper/body_care_products.csv', usecols = cols)
makeup_df = pd.read_csv('../web_scraper/makeup_products.csv', usecols = cols)

Manually merge and drop some categories

In [8]:
skin_care_df['product_category'].replace('Eyes', 'Eye Cream & Treatment', inplace=True)
skin_care_df['product_category'].replace('Eye Mask', 'Eye Cream & Treatment', inplace=True)
skin_care_df['product_category'].replace('Face & Body Sunscreen', 'Sunscreen', inplace=True)
skin_care_df['product_category'].replace('Water-Resistant Sunscreen', 'Sunscreen', inplace=True)
skin_care_df['product_category'].replace('Lips', 'Lip Balm', inplace=True)
skin_care_df['product_category'].replace('AHA Exfoliant', 'Exfoliants', inplace=True)
skin_care_df['product_category'].replace('BHA Exfoliant', 'Exfoliants', inplace=True)
skin_care_df = skin_care_df.loc[skin_care_df['product_category']!='Cleansing Brushes & Devices']
skin_care_df.drop_duplicates(inplace=True)

In [9]:
makeup_df['product_category'].replace('Eyeshadow Palette','Eyeshadow',inplace=True)
makeup_df['product_category'].replace('Waterproof Mascara','Mascara',inplace=True)
makeup_df['product_category'].replace('Foundation Without Sunscreen','Foundation',inplace=True)
makeup_df['product_category'].replace('Foundation With Sunscreen','Foundation',inplace=True)
makeup_df['product_category'].replace('BB & CC Cream','Foundation',inplace=True)
makeup_df['product_category'].replace('Tinted Moisturizer','Foundation',inplace=True)
makeup_df = makeup_df.loc[makeup_df['product_category']!='Makeup Brushes']
makeup_df.drop_duplicates(inplace=True)

#### Before processing all products, let's sample a few products and check how the data cleaning is doing..

check basic data cleaning:

In [10]:
sample = skin_care_df.sample(10)
data_cleaner = product_df_cleaning(sample)
data_cleaner.basic_clean()
sample_cleaned = data_cleaner.get_df()
sample_cleaned

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,product_names,product_category,brand,ingredient,size,price,size_num,size_unit,avg_price,active_ingredient,inactive_ingredient,active_ingredient_list,inactive_ingredient_list,n_inactive_ingredient,n_active_ingredient,is_alphabatical
903,Anti-Wrinkle Cleanser,Cleansers,derma e,"Purified Water, Sodium Methyl Cocoyl Taurate ...",6.00 fl. oz.,13.75,177.0,ml,0.077684,,"Purified Water, Sodium Methyl Cocoyl Taurate ...",[],"[Purified Water, Sodium Methyl Cocoyl Taurate ...",15,0,False
469,180º Face Wash,Cleansers,Nu Skin,"Glycerin, Ascorbic Acid, Sesamum Indicum (Sesa...",4.20 fl. oz.,34.40,124.0,ml,0.277419,,"Glycerin, Ascorbic Acid, Sesamum Indicum (Sesa...",[],"[Glycerin, Ascorbic Acid, Sesamum Indicum (Ses...",17,0,False
4054,Hydra-Pure Intense Moisture Cream,Nighttime Moisturizer,Dr. Dennis Gross Skincare,"Water, Cyclomethicone, Glycerin, Sodium PCA...",1.70 fl. oz.,125.00,50.0,ml,2.500000,,"Water, Cyclomethicone, Glycerin, Sodium PCA...",[],"[Water, Cyclomethicone, Glycerin, Sodium PCA, ...",58,0,False
3382,Anew Reversalist Day Renewal Cream SPF 25,Daytime Moisturizer,Avon,"Active Ingredients: Homosalate 7.0%, Octinoxa...",1.70 fl. oz.,32.00,50.0,ml,0.640000,"Homosalate 7.0%, Octinoxate 7.0%, Oxybenzon...","Water/Eau, Glycerin, Dimethicone, Butylene...","[Homosalate 7.0%, Octinoxate 7.0%, Oxybenzone ...","[Water/Eau, Glycerin, Dimethicone, Butylene Gl...",36,4,False
3120,Lip Conditioner,Lip Balm,Caudalie Paris,"Paraffinum Liquidum (Mineral Oil), Ozokerite,...",0.14 fl. oz.,12.00,4.0,ml,3.000000,,"Paraffinum Liquidum (Mineral Oil), Ozokerite,...",[],"[Paraffinum Liquidum (Mineral Oil), Ozokerite,...",25,0,False
1991,Eye Bright,Eye Cream & Treatment,DHC,"Water, Cucumis Sativus (Cucumber) Juice, But...",0.52 fl. oz.,23.00,15.0,ml,1.533333,,"Water, Cucumis Sativus (Cucumber) Juice, But...",[],"[Water, Cucumis Sativus (Cucumber) Juice, Buty...",14,0,False
3875,Protini Polypeptide Cream,Nighttime Moisturizer,Drunk Elephant,"Water/Aqua/Eau, Dicaprylyl Carbonate, Glycer...",1.69 fl. oz.,68.00,50.0,ml,1.360000,,"Water/Aqua/Eau, Dicaprylyl Carbonate, Glycer...",[],"[Water/Aqua/Eau, Dicaprylyl Carbonate, Glyceri...",58,0,False
3423,Skin Renewing Day Cream with Sunscreen Broad S...,Daytime Moisturizer,CeraVe,"Active: Octinoxate 7.5%, Zinc Oxide 10.5%. In...",1.70 fl. oz.,23.99,50.0,ml,0.479800,"Octinoxate 7.5%, Zinc Oxide 10.5%.","Water, Cyclomethicone, Pentylene Glycol, P...","[Octinoxate 7.5%, Zinc Oxide 10.5%.]","[Water, Cyclomethicone, Pentylene Glycol, Phen...",42,2,False
5143,Lip Rejen Lip & Laugh Line Smoothing Cream,Serum,Skinn by Dimitri James,"Vitis Vinifera (Grape) Fruit Extract, Cartham...",0.50 fl. oz.,36.00,15.0,ml,2.400000,,"Vitis Vinifera (Grape) Fruit Extract, Cartham...",[],"[Vitis Vinifera (Grape) Fruit Extract, Cartham...",16,0,False
4312,Yes to Cucumbers Calming Night Cream,Nighttime Moisturizer,Yes To,"Water, Prunus Amygdalus Dulcis (Sweet Almond)...",1.70 fl. oz.,14.99,50.0,ml,0.299800,,"Water, Prunus Amygdalus Dulcis (Sweet Almond)...",[],"[Water, Prunus Amygdalus Dulcis (Sweet Almond)...",24,0,False


Find all unique ingredients in these 10 products, look up these ingredients in dictionary, check the matching accuracy:

We can see for most ingredient, our matching algorithm find a reasonable match, however, there are a few mistakes:


In [11]:
merged_ingredients = set(list(itertools.chain(*sample_cleaned['inactive_ingredient_list'].values)))
ingredient_property = pd.DataFrame(index=merged_ingredients)
lookup.find_matching_ingredient(merged_ingredients)
ingredient_property['matching'] = [lookup.lookup(ingredient, option='ingredient') for ingredient in merged_ingredients]
ingredient_property['rating'] = [lookup.lookup(ingredient, option='rating') for ingredient in merged_ingredients]
ingredient_property['category'] = [lookup.lookup(ingredient, option='category') for ingredient in merged_ingredients]
ingredient_property

100%|██████████| 233/233 [00:50<00:00,  4.64it/s]


,matching,rating,category
Cetearyl Ethylhexanoate,cetearyl ethylhexanoate,2,"[Texture Enhancer, Emollients]"
Cera Alba (Beeswax),Cera alba,2,[Texture Enhancer]
Sclerocarya Birrea Seed Oil,Sclerocarya birrea,3,"[Emollients, Skin-Replenishing]"
Benzyl Benzoate,retinyl retinoate,3,"[Skin-Restoring, Vitamins]"
Aodium Hyaluronate,sodium hyaluronate,3,"[Skin-Replenishing, Antioxidants, Skin-Restoring]"
Polyvinyl Alcohol,polyvinyl alcohol,1,[Texture Enhancer]
Vitis Vinifera (Grape) Seed Extract,grape seed extract,3,"[Antioxidants, Plant Extracts]"
Cetyl Palmitate,cetyl palmitate,2,[Emollients]
Propylparaben,propylparaben,2,[Preservatives]
Bacillus/Soybean/ Folic Acid Ferment Extract,Vanilla planifolia fruit extract,2,"[Skin-Soothing, Plant Extracts, Antioxidants, ..."


Look up ingredients for these sample product, and see the final cleaned dataframe.

In [12]:
data_cleaner.lookup_ingredients(lookup)
sample_cleaned = data_cleaner.get_df()
sample_cleaned

 12%|█▏        | 28/239 [00:00<00:01, 151.19it/s]

processing all ingredients...


100%|██████████| 239/239 [00:01<00:00, 214.03it/s]
/Users/Qinghui/MachineLearning/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:151: RuntimeWarning: Mean of empty slice
/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


find all ingredients information...


,product_names,product_category,brand,ingredient,size,price,size_num,size_unit,avg_price,active_ingredient,...,inactive_cat_count_Vitamins,active_cat_count_Antioxidants,active_cat_count_Coloring Agents/Pigments,active_cat_count_Skin-Soothing,active_cat_count_Sunscreen Actives,active_cat_count_Thickeners/Emulsifiers,active_mean_rating,inactive_mean_rating,inactive_mean_rating_w1,inactive_mean_rating_w2
903,Anti-Wrinkle Cleanser,Cleansers,derma e,"Purified Water, Sodium Methyl Cocoyl Taurate ...",6.00 fl. oz.,13.75,177.0,ml,0.077684,,...,1.0,0.0,0.0,0.0,0.0,0.0,NaN,2.133333,2.067938,2.103721
469,180º Face Wash,Cleansers,Nu Skin,"Glycerin, Ascorbic Acid, Sesamum Indicum (Sesa...",4.20 fl. oz.,34.40,124.0,ml,0.277419,,...,1.0,0.0,0.0,0.0,0.0,0.0,NaN,2.294118,2.530630,2.375219
4054,Hydra-Pure Intense Moisture Cream,Nighttime Moisturizer,Dr. Dennis Gross Skincare,"Water, Cyclomethicone, Glycerin, Sodium PCA...",1.70 fl. oz.,125.00,50.0,ml,2.500000,,...,3.0,0.0,0.0,0.0,0.0,0.0,NaN,2.275862,2.265043,2.325063
3382,Anew Reversalist Day Renewal Cream SPF 25,Daytime Moisturizer,Avon,"Active Ingredients: Homosalate 7.0%, Octinoxa...",1.70 fl. oz.,32.00,50.0,ml,0.640000,"Homosalate 7.0%, Octinoxate 7.0%, Oxybenzon...",...,1.0,0.0,0.0,0.0,4.0,0.0,3.0,2.083333,2.121380,2.066047
3120,Lip Conditioner,Lip Balm,Caudalie Paris,"Paraffinum Liquidum (Mineral Oil), Ozokerite,...",0.14 fl. oz.,12.00,4.0,ml,3.000000,,...,2.0,0.0,0.0,0.0,0.0,0.0,NaN,2.200000,2.133020,2.204668
1991,Eye Bright,Eye Cream & Treatment,DHC,"Water, Cucumis Sativus (Cucumber) Juice, But...",0.52 fl. oz.,23.00,15.0,ml,1.533333,,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.071429,2.002056,2.000435
3875,Protini Polypeptide Cream,Nighttime Moisturizer,Drunk Elephant,"Water/Aqua/Eau, Dicaprylyl Carbonate, Glycer...",1.69 fl. oz.,68.00,50.0,ml,1.360000,,...,2.0,0.0,0.0,0.0,0.0,0.0,NaN,2.275862,2.266747,2.362563
3423,Skin Renewing Day Cream with Sunscreen Broad S...,Daytime Moisturizer,CeraVe,"Active: Octinoxate 7.5%, Zinc Oxide 10.5%. In...",1.70 fl. oz.,23.99,50.0,ml,0.479800,"Octinoxate 7.5%, Zinc Oxide 10.5%.",...,2.0,1.0,1.0,1.0,1.0,1.0,3.0,2.166667,2.125523,2.198651
5143,Lip Rejen Lip & Laugh Line Smoothing Cream,Serum,Skinn by Dimitri James,"Vitis Vinifera (Grape) Fruit Extract, Cartham...",0.50 fl. oz.,36.00,15.0,ml,2.400000,,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2.250000,2.672146,2.486617
4312,Yes to Cucumbers Calming Night Cream,Nighttime Moisturizer,Yes To,"Water, Prunus Amygdalus Dulcis (Sweet Almond)...",1.70 fl. oz.,14.99,50.0,ml,0.299800,,...,1.0,0.0,0.0,0.0,0.0,0.0,NaN,1.958333,2.092573,2.118415


#### clean up skin care products and save to disk

In [ ]:
data_cleaner = product_df_cleaning(skin_care_df)
data_cleaner.basic_clean()
data_cleaner.lookup_ingredients(lookup)
skin_care_cleaned = data_cleaner.get_df()
skin_care_cleaned.to_csv('skin_care_cleaned.csv',index=False)

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
  0%|          | 0/14190 [00:00<?, ?it/s]

processing all ingredients...


 17%|█▋        | 2361/14190 [09:47<49:03,  4.02it/s]

#### clean up body care products and save to disk

In [ ]:
data_cleaner = product_df_cleaning(body_care_df)
data_cleaner.basic_clean()
data_cleaner.lookup_ingredients(lookup)
body_care_cleaned = data_cleaner.get_df()
body_care_cleaned.to_csv('body_care_cleaned.csv',index=False)

#### clean up make products and save to disk

In [ ]:
data_cleaner = product_df_cleaning(makeup_df)
data_cleaner.basic_clean()
data_cleaner.lookup_ingredients(lookup)
makeup_cleaned = data_cleaner.get_df()
makeup_cleaned.to_csv('makeup_cleaned.csv',index=False)

#### Save matching result to disk in case we want to run this again..

In [ ]:
lookup.save_match_dict()